# **PACKAGE IMPORT**

In [ ]:
!pip install torcheval
# !pip install timm
# !pip install grad-cam

  Obtaining dependency information for torcheval from https://files.pythonhosted.org/packages/e4/de/e7abc784b00de9d05999657d29187f1f7a3406ed10ecaf164de06482608f/torcheval-0.0.7-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.8 MB/s eta 0:00:00a 0:00:01


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from PIL import Image
# Pytorch Imports
from torchvision.datasets import ImageFolder
import torch
from torcheval.metrics.functional import multiclass_f1_score
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, confusion_matrix
from torchvision import models
from albumentations.pytorch import ToTensorV2
from torch.utils.data import random_split
from sklearn.model_selection import StratifiedKFold
import albumentations as A
import copy
from tqdm import tqdm
import timm
# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL
from timm.data.mixup import Mixup
import warnings
warnings.filterwarnings("ignore")
from timm.loss import SoftTargetCrossEntropy
from torch.autograd import Variable


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# **CONFIGURATION**

**You can get the model from [here](https://github.com/huggingface/pytorch-image-models/tree/main/timm/models)**

In [ ]:
# config
path = '/tf'
batch_size = 16
use_amp = True
epochs = 30
lr = 0.0001
num_classes = 6
CLIP_GRAD = 5.0
seed = 42
image_size = 1024
n_fold = 6
fold = 0
model_name = 'convnext_base'
PCH_TEST_DIR = '/tf/train'
WSI_TEST_DIR = '/tf/image_train'
classes = ['CC', 'EC', 'HGSC', 'LGSC', 'MC', 'Normal']
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"
device_ids = list(range(torch.cuda.device_count()))
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(seed)

# **DATA PREPROCESS**

In [ ]:
class UBCDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.file_names = root['file_path'].values
        self.transforms = transforms

        # Assuming your labels are in a column named 'label' in your DataFrame
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.root['label'])

    def __len__(self):
        return len(self.root)

    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]

        if self.transforms:
            img = self.transforms(image=img)["image"]

        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(image_size, image_size),
        A.ShiftScaleRotate(shift_limit=0.1,
                           scale_limit=0.15,
                           rotate_limit=60,
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2,
                sat_shift_limit=0.2,
                val_shift_limit=0.2,
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1),
                contrast_limit=(-0.1, 0.1),
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2()], p=1.),

    "valid": A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2()], p=1.)
}
mixup_fn = Mixup(
        mixup_alpha=0.8, cutmix_alpha=1.0, cutmix_minmax=None,
        prob=0.1, switch_prob=0.5, mode='batch',
        label_smoothing=0.1, num_classes=num_classes)

# **LOAD DATA**

In [ ]:
def get_dataloader(df,fold):
    skf = StratifiedKFold(n_splits=n_fold)
    for fold, ( _, val_) in enumerate(skf.split(X=df, y=df.label)):
          df.loc[val_ , "kfold"] = int(fold)

    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = UBCDataset(df_train, transforms=data_transforms["train"])
    valid_dataset = UBCDataset(df_valid, transforms=data_transforms["valid"])

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader,test_loader

In [ ]:
def get_test_file_path(image_id):
    if os.path.exists(f"{PCH_TEST_DIR}/{image_id}.png"):
        return f"{PCH_TEST_DIR}/{image_id}.png"
    else:
        return f"{WSI_TEST_DIR}/{image_id}.png"
df = pd.read_csv(f"{path}/patch_out.csv")
df['file_path'] = df['image_id'].apply(get_test_file_path)
train_loader,test_loader = get_dataloader(df, fold)

# **MODEL BLOCK**

In [ ]:
# Pooling strategy to replace Max pooling and Average pooling
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)

    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
class UBCModel(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=True, checkpoint_path=None):
        super(UBCModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, checkpoint_path=checkpoint_path)
        in_features = self.model.head.in_features
        self.model.head = nn.Identity()
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear(pooled_features)
#         if self.softmax is not None:
#             output = self.softmax(output)
        return output


model = UBCModel(model_name, num_classes)

# model = models.resnet18()
# num_fcin = model.fc.in_features
# model.fc = nn.Linear(num_fcin, num_classes)

# model = UBCModel(model_name, num_classes)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for training.")
    model = nn.DataParallel(model, device_ids=device_ids)
model.to(device)

UBCModel(
  (model): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
            (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=128, out_features=512, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (norm): Identity()
              (fc2): Linear(in_features=512, out_features=128, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (shortcut): Identity()
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
            (conv_dw): 

# **OPTIMIZER**

In [ ]:
# model = nn.DataParallel(model, device_ids=device_ids)

criterion = SoftTargetCrossEntropy()
val_criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
optimizer = optim.AdamW(model.parameters(),lr=lr)
cosine_schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=20, eta_min=1e-6)

In [ ]:
use_dp=True
if use_amp:
    scaler = torch.cuda.amp.GradScaler()
    if torch.cuda.device_count() > 1 and use_dp:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = torch.nn.DataParallel(model)

# **TRAINING MODULE**

In [ ]:
def train (model,epochs,optimizer,criterion,learing_rate):
    best_epoch_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_accuracy = 0
    best_epoch_loss = 0
    best_epoch_f1_score = 0
    for epoch in range(epochs):
        torch.cuda.empty_cache()
        all_labels = []
        all_predictions = []
        all_labels_val = []
        all_predictions_val = []
        train_epoch_loss = 0.0
        train_class_correct = list(0. for i in range(num_classes))
        train_class_total = list(0. for i in range(num_classes))
        for data in tqdm(train_loader, total=len(train_loader)):

            # probsum = torch.zeros((len(classes),), dtype=torch.float, device=CONFIG["device"])
            image = data['image'].to(device, dtype=torch.float)
            label = Variable(data['label']).to(device, non_blocking=True, dtype=torch.long)

            if len(label) == 2:
                image, label = mixup_fn(image, label)
            # inputs, labels = inputs.to(device), labels.to(device)

            # Clear optimizer graident
            optimizer.zero_grad()

            # Model output of each batch from train_dataset
            output = model(image)
            output = output.to(device)
            label = label.to(device)

            if use_amp:
                with torch.cuda.amp.autocast():
                    batch_loss = torch.nan_to_num(criterion(output, label))
                scaler.scale(batch_loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_GRAD)
                # Unscales gradients and calls
                # or skips optimizer.step()
                scaler.step(optimizer)
                # Updates the scale for next iteration
                scaler.update()
            else:
                batch_loss = criterion(output, label)
                loss.backward()
                # torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_GRAD)
                optimizer.step()

            # 1. Compute train_batch_loss
            torch.cuda.synchronize()
            lr = optimizer.state_dict()['param_groups'][0]['lr']
            train_epoch_loss += batch_loss.item()

            # 2. Compute train_class_correct of each batch
            # print(output)
            _, predicted = torch.max(output, 1)
            if len(label)==2:
                _, label = torch.max(label, 1)
            # print(predicted, label)
            batch_correct = (predicted == label)
            for j in range(len(label)):
                index = label[j].item()
                train_class_correct[index] += batch_correct[j].item()
                train_class_total[index] += 1
            all_labels.append(label.cpu().numpy())
            all_predictions.append(predicted.cpu().numpy())

        # Compute Loss & Acc & F1Score of training
        all_labels = np.concatenate(all_labels)
        all_predictions = np.concatenate(all_predictions)

        train_epoch_loss = train_epoch_loss / len(train_loader)
        train_epoch_accuracy = sum(train_class_correct) / sum(train_class_total) * 100
        train_epoch_f1_score = f1_score(all_labels, all_predictions, average='weighted')
        # train_conf_matrix = confusion_matrix(all_labels, all_predictions)

        print('[Epoch:%2d]' % (epoch + 1))
        print('[Learning rate:%f]' % (lr))
        print('Train Accuracy of All : %.3f %%' % (train_epoch_accuracy))
        print('Train Loss of All : %.3f ' % (train_epoch_loss))
        print('Train F1-Score of All : %.3f ' % (train_epoch_f1_score))
        print("----------------------------------------")

        # Validation class correct & class total
        val_loss = 0.0
        val_class_correct = list(0. for i in range(num_classes))
        val_class_total = list(0. for i in range(num_classes))

        # Validation every epoch
        with torch.no_grad():
            # Run all Validation dataset
            for data in tqdm(test_loader, total=len(test_loader)):
                # probsum = torch.zeros((len(classes),), dtype=torch.float, device=CONFIG["device"])
                image = data['image'].to(device, dtype=torch.float)
                # One batch data
                label = data['label'].to(device, dtype=torch.long)
                # images, labels = images.to(device), labels.to(device)
                # Model output of each batch from Validation_dataset
                output = model(image)
                output = output.to(device)
                label = label.to(device)
                # print(output, label)
                # 1. Compute val_batch_loss
                batch_loss = val_criterion(output, label)
                val_loss += batch_loss.item()

                # 2. Compute val_class_correct of each batch
                _, predicted = torch.max(output, 1)
                batch_correct = (predicted == label).squeeze()
                for j in range(len(label)):
                    index = label[j].item()
                    if len(label) == batch_size:
                        val_class_correct[index] += batch_correct[j].item()
                    else:
                        val_class_correct[index] += batch_correct.item()
                    val_class_total[index] += 1
                all_labels_val.append(label.cpu().numpy())
                all_predictions_val.append(predicted.cpu().numpy())
        # print each class accurancy of  Validation
        for i in range(num_classes):
            label = classes[i]
            print('Accuracy of %5s : %2d %%' % (label, 100 * val_class_correct[i] / val_class_total[i]))

        all_labels_val = np.concatenate(all_labels_val)
        all_predictions_val = np.concatenate(all_predictions_val)

        # Compute Loss & Acc & F1Score of Validation
        val_epoch_accuracy = sum(val_class_correct) / sum(val_class_total) * 100
        val_epoch_loss = val_loss / len(test_loader)
        val_epoch_f1_score = f1_score(all_labels_val, all_predictions_val, average='weighted')
        val_conf_matrix = confusion_matrix(all_labels_val, all_predictions_val)

        print('Validation Accuracy of All : %.3f %%' % (val_epoch_accuracy))
        print('Validation Loss of All : %.3f ' % (val_epoch_loss))
        print('Validation F1-Score of All : %.3f ' % (val_epoch_f1_score))
        print("----------------------------------------")

        # store the best weights
        if best_epoch_accuracy < val_epoch_accuracy:
            print(f"{b_}Validation Accuracy Improved ({best_epoch_accuracy} ---> {val_epoch_accuracy})")
            print(f"{b_}Validation Loss Changed ({best_epoch_loss} ---> {val_epoch_loss})")
            print(f"{b_}Validation F1-Score Changed ({best_epoch_f1_score} ---> {val_epoch_f1_score})")
            best_epoch_accuracy = val_epoch_accuracy
            best_epoch_loss = val_epoch_loss
            best_epoch_f1_score = val_epoch_f1_score
            best_epoch_model_wts = copy.deepcopy(model.state_dict())
            PATH = "{:}_Acc{:.2f}_Loss{:.4f}_F1-Score{:.4f}_epoch{:.2f}.bin".format(model_name, best_epoch_accuracy, val_epoch_loss, val_epoch_f1_score, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")

            # torch.save(model.state_dict(), '/tf/model.pth')
        cosine_schedule.step()
    print('Finished Training')


# **TRAINING**

In [ ]:
train(model,epochs,optimizer,criterion,lr)

100%|██████████| 216/216 [33:35<00:00,  9.33s/it]


[Epoch: 1]
[Learning rate:0.000100]
Train Accuracy of All : 41.067 %
Train Loss of All : 1.645 
Train F1-Score of All : 0.343 
----------------------------------------


100%|██████████| 54/54 [00:22<00:00,  2.44it/s]


Accuracy of    CC :  0 %
Accuracy of    EC :  0 %
Accuracy of  HGSC : 100 %
Accuracy of  LGSC :  0 %
Accuracy of    MC :  0 %
Validation Accuracy of All : 41.121 %
Validation Loss of All : 1.446 
Validation F1-Score of All : 0.240 
----------------------------------------
Validation Accuracy Improved (0 ---> 41.1214953271028)
Validation Loss Changed (0 ---> 1.445639889549326)
Validation F1-Score Changed (0 ---> 0.2396484495884137)
Model Saved


100%|██████████| 216/216 [33:31<00:00,  9.31s/it]


...

100%|██████████| 216/216 [33:38<00:00,  9.35s/it]


[Epoch:21]
[Learning rate:0.000001]
Train Accuracy of All : 97.216 %
Train Loss of All : 0.647 
Train F1-Score of All : 0.972 
----------------------------------------


100%|██████████| 54/54 [00:22<00:00,  2.42it/s]


Accuracy of    CC : 75 %
Accuracy of    EC : 44 %
Accuracy of  HGSC : 88 %
Accuracy of  LGSC : 44 %
Accuracy of    MC : 66 %
Validation Accuracy of All : 70.093 %
Validation Loss of All : 0.907 
Validation F1-Score of All : 0.691 
----------------------------------------


 12%|█▏        | 25/216 [03:55<29:47,  9.36s/it]